## Regressao Logistica e Linear

In [39]:
# Análise do desempenho dos estudantes de português do ensino médio de Portugal

# --- 1. Importação de bibliotecas ---
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.metrics import r2_score, mean_squared_error, accuracy_score, precision_score, recall_score, confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
# --- 2. Carregamento dos dados ---
portugues = pd.read_csv('https://docs.google.com/spreadsheets/d/e/2PACX-1vQmEm4SuBKE2_dXddpxAM38P7YbLWcRIAcb5lqql9nKlw4pbNd4sZS__a6oK1mI3MDfQu8PvHEUJVMC/pub?gid=998950341&single=true&output=csv')
# --- 3. Tratamento de valores nulos ---
for col in portugues.columns:
    if portugues[col].dtype == 'object':
        portugues[col] = portugues[col].fillna('não informado')
    else:
        portugues[col] = portugues[col].fillna(portugues[col].median())

# --- 4. Pré-processamento ---
def preprocessamento(df):
    df_proc = df.copy()
    le = LabelEncoder()
    for col in df_proc.columns:
        if df_proc[col].dtype == 'object':
            df_proc[col] = le.fit_transform(df_proc[col].astype(str))
    df_proc = df_proc.fillna(0)
    return df_proc
portugues_proc = preprocessamento(portugues)

### --- 5. Análise e Modelagem: Matemática ---
print("\n===== ANÁLISE - Português =====")

# Regressão
X_por = portugues_proc.drop(['nota_final'], axis=1)
y_por = portugues_proc['nota_final']
X_train_m, X_test_m, y_train_m, y_test_m = train_test_split(X_por, y_por, test_size=0.3, random_state=42)

lr = LinearRegression()
lr.fit(X_train_m, y_train_m)
y_pred_lr = lr.predict(X_test_m)

print("\nRegressão Linear - Português sem PCA")
print("R²:", r2_score(y_test_m, y_pred_lr))
print("RMSE:", np.sqrt(mean_squared_error(y_test_m, y_pred_lr)))

# Classificação
portugues_proc['Aprovado'] = np.where(portugues_proc['nota_final'] >= 10, 1, 0)
X_class = portugues_proc.drop(['nota_final', 'Aprovado'], axis=1)
y_class = portugues_proc['Aprovado']
X_train_c, X_test_c, y_train_c, y_test_c = train_test_split(X_class, y_class, test_size=0.3, random_state=42)

logr = LogisticRegression(max_iter=1000)
logr.fit(X_train_c, y_train_c)
y_pred_logr = logr.predict(X_test_c)

print("\nRegressão Logística - Português sem PCA")
print("Acurácia:", accuracy_score(y_test_c, y_pred_logr))
print("Precisão:", precision_score(y_test_c, y_pred_logr))
print("Recall:", recall_score(y_test_c, y_pred_logr))
print("Matriz de Confusão:\n", confusion_matrix(y_test_c, y_pred_logr))

# Padronizando os dados (muito importante antes de aplicar PCA)
escalador = StandardScaler()
portugues_padronizados = escalador.fit_transform(X_por)

# Definindo o PCA para manter 95% da variância
pca = PCA(n_components=0.95)  # Isso diz: mantenha 95% da informação

# Aplicando o PCA
portugues_pca = pca.fit_transform(portugues_padronizados)

# Ver quantos componentes foram escolhidos automaticamente
print(f"Número de componentes escolhidos para manter 95% da variância: {pca.n_components_}")

# Variância explicada por cada componente
print("\nVariância explicada por cada componente:")
print(pca.explained_variance_ratio_)

# Criando DF dos dados reduzidos
df_portugues_pca = pd.DataFrame(data=portugues_pca, columns=[f'Componente {i+1}' for i in range(pca.n_components_)])

# Aplicando regressão linear sobre a base reduzida
# Regressão
X_por_pca = df_portugues_pca
y_por = portugues_proc['nota_final']
X_train_m, X_test_m, y_train_m, y_test_m = train_test_split(X_por_pca, y_por, test_size=0.3, random_state=42)

lr = LinearRegression()
lr.fit(X_train_m, y_train_m)
y_pred_lr = lr.predict(X_test_m)

print("\nRegressão Linear - Português com PCA")
print("R²:", r2_score(y_test_m, y_pred_lr))
print("RMSE:", np.sqrt(mean_squared_error(y_test_m, y_pred_lr)))


# Classificação
X_class_pca = df_portugues_pca
y_class = portugues_proc['Aprovado']
X_train_c, X_test_c, y_train_c, y_test_c = train_test_split(X_class_pca, y_class, test_size=0.3, random_state=42)

logr = LogisticRegression(max_iter=1000)
logr.fit(X_train_c, y_train_c)
y_pred_logr = logr.predict(X_test_c)

print("\nRegressão Logística - Português com PCA")
print("Acurácia:", accuracy_score(y_test_c, y_pred_logr))
print("Precisão:", precision_score(y_test_c, y_pred_logr))
print("Recall:", recall_score(y_test_c, y_pred_logr))
print("Matriz de Confusão:\n", confusion_matrix(y_test_c, y_pred_logr))


===== ANÁLISE - Português =====

Regressão Linear - Português sem PCA
R²: 0.8699049941859338
RMSE: 1.1994205111292888

Regressão Logística - Português sem PCA
Acurácia: 0.9230769230769231
Precisão: 0.9529411764705882
Recall: 0.9585798816568047
Matriz de Confusão:
 [[ 18   8]
 [  7 162]]
Número de componentes escolhidos para manter 95% da variância: 28

Variância explicada por cada componente:
[0.11961441 0.07975704 0.05542864 0.04735767 0.04451793 0.04092949
 0.03961165 0.03940761 0.03509851 0.03337025 0.03222352 0.031072
 0.02991894 0.02912885 0.02846884 0.02724144 0.02678849 0.02542356
 0.02294465 0.0225712  0.02190257 0.02096518 0.02061161 0.01988259
 0.019337   0.01784553 0.01583322 0.01512621]

Regressão Linear - Português com PCA
R²: 0.8340860107745249
RMSE: 1.354511391276687

Regressão Logística - Português com PCA
Acurácia: 0.9230769230769231
Precisão: 0.9529411764705882
Recall: 0.9585798816568047
Matriz de Confusão:
 [[ 18   8]
 [  7 162]]


In [41]:
# Análise do desempenho dos estudantes de Matematica do ensino médio de Portugal

# --- 1. Importação de bibliotecas ---
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.metrics import r2_score, mean_squared_error, accuracy_score, precision_score, recall_score, confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
# --- 2. Carregamento dos dados ---
matematica = pd.read_csv('/home/desktop_dev/Documentos/GitHub/BOOTCAMP_LLM/Machine Learning/dados/Desempenho.csv')
# --- 3. Tratamento de valores nulos ---
for col in matematica.columns:
    if matematica[col].dtype == 'object':
        matematica[col] = matematica[col].fillna('não informado')
    else:
        matematica[col] = matematica[col].fillna(matematica[col].median())

# --- 4. Pré-processamento ---
def preprocessamento(df):
    df_proc = df.copy()
    le = LabelEncoder()
    for col in df_proc.columns:
        if df_proc[col].dtype == 'object':
            df_proc[col] = le.fit_transform(df_proc[col].astype(str))
    df_proc = df_proc.fillna(0)
    return df_proc
matematica_proc = preprocessamento(matematica)

### --- 5. Análise e Modelagem: Matemática ---
print("\n===== ANÁLISE - Matematica =====")

# Regressão
X_por = matematica_proc.drop(['nota_final'], axis=1)
y_por = matematica_proc['nota_final']
X_train_m, X_test_m, y_train_m, y_test_m = train_test_split(X_por, y_por, test_size=0.3, random_state=42)

lr = LinearRegression()
lr.fit(X_train_m, y_train_m)
y_pred_lr = lr.predict(X_test_m)

print("\nRegressão Linear - Matematica sem PCA")
print("R²:", r2_score(y_test_m, y_pred_lr))
print("RMSE:", np.sqrt(mean_squared_error(y_test_m, y_pred_lr)))

# Classificação
matematica_proc['Aprovado'] = np.where(matematica_proc['nota_final'] >= 10, 1, 0)
X_class = matematica_proc.drop(['nota_final', 'Aprovado'], axis=1)
y_class = matematica_proc['Aprovado']
X_train_c, X_test_c, y_train_c, y_test_c = train_test_split(X_class, y_class, test_size=0.3, random_state=42)

logr = LogisticRegression(max_iter=1000)
logr.fit(X_train_c, y_train_c)
y_pred_logr = logr.predict(X_test_c)

print("\nRegressão Logística - Matematica sem PCA")
print("Acurácia:", accuracy_score(y_test_c, y_pred_logr))
print("Precisão:", precision_score(y_test_c, y_pred_logr))
print("Recall:", recall_score(y_test_c, y_pred_logr))
print("Matriz de Confusão:\n", confusion_matrix(y_test_c, y_pred_logr))

# Padronizando os dados (muito importante antes de aplicar PCA)
escalador = StandardScaler()
matematica_padronizados = escalador.fit_transform(X_por)

# Definindo o PCA para manter 95% da variância
pca = PCA(n_components=0.95)  # Isso diz: mantenha 95% da informação

# Aplicando o PCA
matematica_pca = pca.fit_transform(matematica_padronizados)

# Ver quantos componentes foram escolhidos automaticamente
print(f"Número de componentes escolhidos para manter 95% da variância: {pca.n_components_}")

# Variância explicada por cada componente
print("\nVariância explicada por cada componente:")
print(pca.explained_variance_ratio_)

# Criando DF dos dados reduzidos
df_matematica_pca = pd.DataFrame(data=matematica_pca, columns=[f'Componente {i+1}' for i in range(pca.n_components_)])

# Aplicando regressão linear sobre a base reduzida
# Regressão
X_por_pca = df_matematica_pca
y_por = matematica_proc['nota_final']
X_train_m, X_test_m, y_train_m, y_test_m = train_test_split(X_por_pca, y_por, test_size=0.3, random_state=42)

lr = LinearRegression()
lr.fit(X_train_m, y_train_m)
y_pred_lr = lr.predict(X_test_m)

print("\nRegressão Linear - Matematica com PCA")
print("R²:", r2_score(y_test_m, y_pred_lr))
print("RMSE:", np.sqrt(mean_squared_error(y_test_m, y_pred_lr)))


# Classificação
X_class_pca = df_matematica_pca
y_class = matematica_proc['Aprovado']
X_train_c, X_test_c, y_train_c, y_test_c = train_test_split(X_class_pca, y_class, test_size=0.3, random_state=42)

logr = LogisticRegression(max_iter=1000)
logr.fit(X_train_c, y_train_c)
y_pred_logr = logr.predict(X_test_c)

print("\nRegressão Logística - Matematica com PCA")
print("Acurácia:", accuracy_score(y_test_c, y_pred_logr))
print("Precisão:", precision_score(y_test_c, y_pred_logr))
print("Recall:", recall_score(y_test_c, y_pred_logr))
print("Matriz de Confusão:\n", confusion_matrix(y_test_c, y_pred_logr))


===== ANÁLISE - Matematica =====

Regressão Linear - Matematica sem PCA
R²: 0.7947708519011206
RMSE: 2.1239962683180083

Regressão Logística - Matematica sem PCA
Acurácia: 0.9243697478991597
Precisão: 0.9444444444444444
Recall: 0.9315068493150684
Matriz de Confusão:
 [[42  4]
 [ 5 68]]
Número de componentes escolhidos para manter 95% da variância: 27

Variância explicada por cada componente:
[0.10761625 0.07815344 0.06108764 0.05554474 0.04759677 0.04418075
 0.04344667 0.0387702  0.03717492 0.03407454 0.03317383 0.03182103
 0.03002372 0.02941724 0.02811099 0.02631177 0.02502711 0.02461771
 0.0237259  0.02236053 0.02198045 0.01985613 0.01953185 0.01852682
 0.01748344 0.01651775 0.01533431]

Regressão Linear - Matematica com PCA
R²: 0.7515811101292525
RMSE: 2.336826863564151

Regressão Logística - Matematica com PCA
Acurácia: 0.9159663865546218
Precisão: 0.9315068493150684
Recall: 0.9315068493150684
Matriz de Confusão:
 [[41  5]
 [ 5 68]]


/home/desktop_dev/Documentos/GitHub/BOOTCAMP_LLM/.venv/lib/python3.12/site-packages/numpy/lib/_nanfunctions_impl.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
